In [4]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 512kB 39.6MB/s 
     |████████████████████████████████| 3.8MB 35.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=96b1ed5461071b2d243ee21b34e855be6e67a8f280f72ce367193cc8aae0fde8
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: -ensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: -ensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: -ensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from skimage import transform
def resize_batch(imgs):
    # A function to resize a batch of MNIST images to (32, 32)
    # Args:
    #     imgs: a numpy array of size [batch_size, 28 X 28].
    # Returns:
    #     a numpy array of size [batch_size, 32, 32].
    imgs = imgs.reshape((-1, 28, 28, 1))
    resized_imgs = np.zeros((imgs.shape[0], 32, 32, 1))
    for i in range(imgs.shape[0]):
        resized_imgs[i, ..., 0] = transform.resize(imgs[i, ..., 0], (32, 32))

    return resized_imgs

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(dtype=tf.float32, shape=[None,32, 32, 1])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
Kernel1 = tf.Variable(tf.random_normal([5,5,1,6], stddev=0.01))
conv2d_1=tf.nn.conv2d(X, Kernel1, strides=[1,1,1,1], padding='VALID')
conv2d_1=tf.nn.relu(conv2d_1)
conv2d_1=tf.nn.max_pool(conv2d_1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

Kernel2 = tf.Variable(tf.random_normal([5,5,6,16], stddev=0.01))
conv2d_2=tf.nn.conv2d(conv2d_1,Kernel2, strides=[1,1,1,1], padding='VALID')
conv2d_2=tf.nn.relu(conv2d_2)
conv2d_2=tf.nn.max_pool(conv2d_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
#conv2d_2=tf.reshape(conv2d_2, [-1,-5*5*16])
conv2d_2=tf.contrib.layers.flatten(conv2d_2)


W1 = tf.Variable(tf.random_normal([400, 120]))
b1 = tf.Variable(tf.random_normal([120]))
Z1 = tf.nn.sigmoid(tf.matmul(conv2d_2, W1) + b1)

W2 = tf.Variable(tf.random_normal([120 , 84]))
b2 = tf.Variable(tf.random_normal([84]))
Z2 = tf.nn.sigmoid(tf.matmul(Z1, W2) + b2)

W3 = tf.Variable(tf.random_normal([84 ,10]))
b3 = tf.Variable(tf.random_normal([10]))
S = tf.nn.sigmoid(tf.matmul(Z2, W3) + b3)

hypothesis = S
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
corrects = tf.equal(tf.argmax(hypothesis, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))

LEARNING_RATE = 0.001
EPOCHS = 15
BATCH_SIZE = 100
optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(EPOCHS):
        for _ in range(int(mnist.train.num_examples / BATCH_SIZE)):
            X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
            X_batch = resize_batch(X_batch)
            sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})

        if (epoch+1) % 1 == 0:
            acc_val = sess.run(accuracy,
                feed_dict={X: resize_batch(mnist.test.images),
                    Y: mnist.test.labels})
            print("epoch: {:04d}, accuracy: {:.4f}".format(epoch+1, acc_val))

    print("Learning finished.")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr